In [42]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez

In [43]:
Entrez.email = 'greg.clinton@gmail.com'

In [ ]:
# How many genes does e. coli have? What are their names?
# How many species under homo?
# sra vs nucleotide, gene, ref_seq

In [44]:
record = Entrez.read(Entrez.esearch(db = 'gene', term = 'e coli[Orgn]'))

In [49]:
ids = ','.join(record['IdList'][:5])
ids

'947170,948466,947777,948511,947071'

In [50]:
records = Entrez.read(Entrez.efetch(db = 'gene', id = ids, retmode = 'xml'))
len(records)

5

In [52]:
print records[0].keys()

[u'Entrezgene_rna', u'Entrezgene_unique-keys', u'Entrezgene_track-info', u'Entrezgene_comments', u'Entrezgene_locus', u'Entrezgene_source', u'Entrezgene_type', u'Entrezgene_gene-source', u'Entrezgene_xtra-iq', u'Entrezgene_non-unique-keys', u'Entrezgene_gene', u'Entrezgene_properties']


In [58]:
print records[0]['Entrezgene_type']

2


In [64]:
record = Entrez.read(Entrez.esearch(db = 'nuccore', term = 'e coli[Orgn]'))
ids = ','.join(record['IdList'][:5])
handle = Entrez.efetch(db = 'nuccore', id = ids, rettype = 'gb', retmode = 'text')
print handle.read()

LOCUS       AH004803                1143 bp    DNA     linear   BCT 26-AUG-2016
DEFINITION  Escherichia coli fanF, fanG, and fanE genes, complete sequence.
ACCESSION   AH004803 S79577 S79584
VERSION     AH004803.2  GI:1060041767
KEYWORDS    .
SOURCE      Escherichia coli
  ORGANISM  Escherichia coli
            Bacteria; Proteobacteria; Gammaproteobacteria; Enterobacteriales;
            Enterobacteriaceae; Escherichia.
REFERENCE   1  (bases 1 to 1143)
  AUTHORS   Lee,J.H. and Isaacson,R.E.
  TITLE     Expression of the gene cluster associated with the Escherichia coli
            pilus adhesin K99
  JOURNAL   Infect. Immun. 63 (10), 4143-4149 (1995)
   PUBMED   7558331
COMMENT     On or before Aug 26, 2016 this sequence version replaced
            gi:1168149, gi:1168150, gi:1683568.
FEATURES             Location/Qualifiers
     source          1..1143
                     /organism="Escherichia coli"
                     /mol_type="genomic DNA"
                     /db_xref="taxon:56

In [63]:
handle.read()

''